In [ ]:
import os
import pygmt
import pandas as pd
import xarray as xr
import numpy as np
from scripts import mapOutputs as mout

import matplotlib.pyplot as plt
%matplotlib inline

## Running the simulations

To run the different simulations, you will need to do the following in a terminal:

```bash
mpirun -np X python3 runModel.py -i input-global_refined.yml 
```

where X is the number of processors to use (for example 10).

## Visualising the outputs

You can visualise the outputs of your simulations directly in Paraview. Below we also show how this could be extracted as netcdf grids and analysed in Python.


To do so, we will be using the `mapOutputs` Python class (available in the `scripts` folder) to first extract the outputs, remap the unstructured variables on a structured mesh. The class is also performing additional calculations on the dataset to extract individual catchments based on flow accumulation and elevation.

The interpolated model's variables are then exported as `netCDF` files, and each of them contains the following (if available from the simulation):

+ elevation `elevation` (m)
+ cumulative erosion/deposition `erodep` (m)
+ erosion/deposition rate `erodep_rate` (m/yr)
+ water flux discharge (accounting for lakes) `fillDischarge`(m3/yr)
+ sediment flux in rivers `sedimentLoad` (m3/yr)
+ main basin ids `basinID` (integer)
+ rainfall `precipitation` (m/yr)
+ tectonic `uplift` (m/yr)
+ cumulative flexural isostasy `flex` (m)
  

We first define a folder where exported files will be stored:

In [ ]:
# Define output folder name for the simulation
out_path = 'export'

if not os.path.exists(out_path):
    os.makedirs(out_path)

Then we will be using the `mapOutputs` Python class to first extract the outputs, remap the unstructured variables on a structured mesh (the resolution of this mesh is defined by the `reso` parameter set in degree). The class is also performing additional calculations on the dataset to extract individual catchments based on flow accumulation and elevation.

The interpolated model's variables are then exported as `netCDF` files, and each of them contains the following:

+ elevation `elevation` (m)
+ cumulative erosion/deposition `erodep` (m)
+ water flux discharge (accounting for lakes) `fillDischarge`(m3/yr)
+ water flux in rivers (not showing lakes) `flowDischarge` (m3/yr)
+ sediment flux in rivers `sedimentLoad` (m3/yr)
+ main basin ids `basinID` (integer)
+ rainfall `precipitation` (m/yr)
+ tectonic `uplift` (m/yr)

In [ ]:
stp = 0

# Resolution of the netcdf structured grid
reso = 0.1

# Name of each netcdf output file
ncout = os.path.join(out_path, "data")

# Initialisation of the class
grid = mout.mapOutputs(path='./', filename='input-mardep.yml', step=stp, uplift=False, flex=True)

We perform the export using the following loop:

In [ ]:
for k in range(0,11):
    
    if stp>1:
        # Get goSPL variables
        grid.getData(stp)
        
    # Remap the variables on the regular mesh using distance weighting interpolation
    grid.buildLonLatMesh(res=reso, nghb=3)
    
    # Export corresponding regular mesh variables as netCDF file
    grid.exportNetCDF(ncfile = ncout+str(k)+'.nc')
    stp += 1

# Plotting some outputs

In [ ]:
dataset1 = xr.open_dataset(out_path+'/data1.nc')
dataset5 = xr.open_dataset(out_path+'/data5.nc')
dataset10 = xr.open_dataset(out_path+'/data10.nc')

### Mollweide projection

In [ ]:
fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="geo", series=[-10000, 10000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset10.elevation, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a2000", "x+lElevation", "y+lm"])
# Customising the font style
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)

In [ ]:
fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="polar", series=[-3000, 3000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset10.erodep, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a1000", "x+lErosion/Deposition", "y+lm"])
# Customising the font style
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)

In [ ]:
dataset10

In [ ]:
fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="polar", series=[-5000, 5000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset10.flex, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a1000", "x+lFlexure", "y+lm"])
# Customising the font style
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)

# Evaluation of major drainage systems

Here we design a workflow to extract for each time step the characteristics of each catchment in terms of sediment and water fluxes. The operation is performed globally.

> To avoid having to read the large netcdf file at each time step, we first create a new set of netcdf files with only required variables for the analysis 

In [ ]:
# Specify the folder containing the netcdf file from our simulation
out = 'export/'

# Loop over each file and drop unwanted variables
for k in range(0,11,1):
    
    # Open the netcdf file
    dataset = xr.open_dataset(out+'data'+str(k)+'.nc')
    
    # Drop some variables (we only keep the sediment flow fluxes and the basin indices)
    reduce_ds = dataset[['flowDischarge','sedimentLoad','basinID']]
    
    # Save the reduced dataset as a new smaller netcdf
    reduce_ds.to_netcdf(out+'fsdata'+str(k)+'.nc')

We can then use the function `getCatchmentInfo.py` to extract for each individual catchment the maximum fluxes. 

This operation especially when performed globally and over all time steps is slow. The good news is that it is pretty much embarrassingly parallel so we will take advantage of this.

If you want to run it, you can uncomment the following line and run it on 8 cpus (if you actually have 8 cpus on your machine)

The script takes an input file `inputSedFlow.csv` that contains 2 columns: 

+ the first one if the time in Ma and
+ the second is the netcdf file (created above) for each specified time interval.

In [ ]:
# !mpirun -np 9 python3 getCatchmentInfo.py -i inputSedFlow.csv -o flowsed

# Extract fluxes (sediment and flow)

Once finished, you will have 2 `csv` files per time step (one for the water discharge and one for the sediment fluxes). Each file contains 4 columns:

+ the first one is the basin index,
+ the second and third ones are the position of the outlet (longitude and latitude respectively),
+ the last column is the corresponding discharge or flux value.

In [ ]:
# Pick a time in Ma needs to be a integer
step = 10

# Get the fluxes files 
flowdf = pd.read_csv('flowsed/flow'+str(step)+'.csv')
seddf = pd.read_csv('flowsed/sed'+str(step)+'.csv')

We will use the logarithmic values 

In [ ]:
logFA = np.log10(flowdf['val'].values)
logSed = np.log10(seddf['val'].values)

Let's say we want to take the nth-largest values. To do so, we will first sort the values

In [ ]:
sorted_index_array = np.argsort(logFA)
sortedFA = logFA[sorted_index_array]
sortedLon = flowdf['lon'].values[sorted_index_array]
sortedLat = flowdf['lat'].values[sorted_index_array]

sorted_index_array1 = np.argsort(logSed)
sortedSed = logSed[sorted_index_array1]
sortedLon1 = seddf['lon'].values[sorted_index_array1]
sortedLat1 = seddf['lat'].values[sorted_index_array1]

We now want to visualise the main fluxes, for example we will look at the 200 largest fluxes:

In [ ]:
# Define nth values
nlargest = 200

rLon = sortedLon[-nlargest : ]
rLat = sortedLat[-nlargest : ]
rFA = sortedFA[-nlargest : ]

rLon1 = sortedLon1[-nlargest : ]
rLat1 = sortedLat1[-nlargest : ]
rSed = sortedSed[-nlargest : ]

### Plot for flow

In [ ]:
# Let read the initial dataset (this will be used to get the elevation on our plot)
dataset = xr.open_dataset(out+'data'+str(step)+'.nc')

fig = pygmt.Figure()
# Background image
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="gray", series=[-6000, 6000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset.elevation, shading='+a345+nt1+m0', frame=False)
    
    fig.grdcontour(
        interval=0.1,
        grid=dataset.elevation,
        limit=[-0.1, 0.1],
    )
# Scatter plot
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="devon", series=[rFA.min(), rFA.max()], reverse=True)
    fig.plot(
        x=rLon,
        y=rLat,
        style="cc",
        pen="white",
        size=0.00005 * 2 ** rFA,
        fill=rFA,
        cmap=True,
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h", 
                 frame="af+l'Water fluxes (log-scale) (m3/yr)'")
# Time interval
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black")

fig.show(dpi=500, width=1000)
# fname = 'fluxes/flow'+str(step)+'Ma.png'
# fig.savefig(fname=fname,dpi=500)

### Plot for sed

In [ ]:
fig = pygmt.Figure()

# Background image
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="gray", series=[-6000, 6000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset.elevation, shading='+a345+nt1+m0', frame=False)
    fig.grdcontour(
        interval=0.1,
        grid=dataset.elevation,
        limit=[-0.1, 0.1],
    )
# Scatter plot
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="buda", series=[rSed.min(), rSed.max()])
    fig.plot(
        x=rLon1,
        y=rLat1,
        style="cc",
        pen="black",
        size=0.0005 * 2 ** rSed,
        fill=rSed,
        cmap=True,
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h", 
                 frame="af+l'Sediment fluxes (log-scale) (m3/yr)'")
# Time interval
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black")
fig.show(dpi=500, width=1000)
# fname = 'fluxes/sed'+str(step)+'Ma.png'
# fig.savefig(fname=fname,dpi=500)